<style>div.container { width: 100% }</style>
<img style="float:left;  vertical-align:text-bottom;" height="65" width="172" src="../../assets/holoviz-logo-unstacked.svg" />
<div style="float:right; vertical-align:text-bottom;"><h2>Exercise 1: Plotting</h2></div>

In this exercise we will try making some of our own visualizations using hvplot.

#### Loading the data 

We will be building a new visualization based on the same data we have cleaned and filtered in the rest of the tutorial. First we load the earthquakes `DataFrame` and filter to those with `>=7` magnitude:

In [ ]:
import pathlib
import pandas as pd 
import xarray as xr

import hvplot.pandas # noqa: adds hvplot method to pandas objects
import hvplot.xarray # noqa: adds hvplot method to xarray objects

df = pd.read_parquet(pathlib.Path('../../data/earthquakes-projected.parq'))
df = df.set_index(df.time)
most_severe = df[df.mag >= 7]

And next we load the population density raster data:

In [ ]:
ds = xr.open_dataarray(pathlib.Path('../../data/raster/gpw_v4_population_density_rev11_2010_2pt5_min.nc'))
cleaned_ds = ds.where(ds.values != ds.nodatavals).sel(band=1)
cleaned_ds.name = 'population'

#### Visualizing the raster data

Start by using `hvplot.image` to visualize the whole of the population density data using the Datashader support in hvPlot. Grab a handle on this `Image` HoloViews object called `pop_density` and customize it in the `.hvplot.image` call, enabling a logarithmic color scale and a blue colormap (specifically the `'Blues'` colormap). At the end of the cell, display this object.

<br><details><summary>Hint</summary><br>

Don't forget to include `rasterize=True` in the `hvplot.image` call.  You can also use `logz=True`, `clim=(1, np.nan)` and `cmap='Blues'`.

</details>

In [ ]:
pop_density = ... # Use hvplot here to visualize the data in cleaned_ds
pop_density  # Display it

<details><summary>Solution</summary><br>

```python
pop_density = cleaned_ds.hvplot.image(rasterize=True, logz=True, clim=(1, np.nan), cmap='Blues') 
pop_density
```

<br></details>

#### Visualizing the earthquake positions

Now visualize the tabular data in `most_severe` by building a `hv.Points` object with `.hvplot`. This will be very similar to the approach shown in the tutorial but this time we want all the earthquakes to be marked with red crosses of size 100. As above, get a handle on the resulting object and display it where the handle is now called `quake_points`.

_Hint: To figure out which arguments you need to pass, try `help(most_severe.hvplot.points)`, then if you can find the right option for marking the points but don't know what value to pass, try passing an invalid value to see from the error message what the valid options are._

<br><details><summary>Hint</summary><br>

Don't forget to map the longitude and latitude dimensions to `x` and `y` in the call to `hvplot.points`.

</details>

In [ ]:
quake_points = ... # Use hvplot here to visualize the data in most_severe 
quake_points

<details><summary>Solution</summary><br>

```python
quake_points = most_severe.hvplot.points(x='longitude', y='latitude', marker='+', size=100, color='red')
quake_points 
```

<br></details>

#### Composing these visualizations together

Now overlay `quake_points` over `pop_density` to check everything looks correct. Make sure the box select tool is working as expected.

<details><summary>Solution</summary><br>
    
```python
pop_density * quake_points
```

<br></details>